# Converter for the Financial Filings at the Sec

In [31]:
from zipfile import ZipFile
import pandas as pd

with ZipFile('importFiles/2021q4.zip') as myzip:
    with myzip.open('num.txt') as myfile1:
        dfNum = pd.read_table(myfile1,delimiter="\t")
    with myzip.open('pre.txt') as myfile2:
        dfPre = pd.read_table(myfile2,delimiter="\t")
    with myzip.open('sub.txt') as myfile3:
        dfSub = pd.read_table(myfile3,delimiter="\t", low_memory=False)
    with myzip.open('tag.txt') as myfile4:
        dfTag = pd.read_table(myfile4,delimiter="\t")

dfSym = pd.read_table('importFiles/ticker.txt', delimiter="\t", header=None, names=['symbol','cik'])        
dirname = "2021q4"

print(dfNum.size)
print(dfPre.size)
print(dfSub.size)
print(dfTag.size)
print(dfSym.size)

24282297
13311870
867312
1083789
24168


In [32]:
from tabulate import tabulate

print(tabulate(dfNum.head(100), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+----------------------+---------------+--------------+---------+----------+--------+-------+--------------+------------+
|    | adsh                 | tag           | version      |   coreg |    ddate |   qtrs | uom   |        value |   footnote |
|----+----------------------+---------------+--------------+---------+----------+--------+-------+--------------+------------|
|  0 | 0000008858-21-000048 | NetIncomeLoss | us-gaap/2020 |     nan | 20200930 |      1 | USD   | -1.8889e+07  |        nan |
|  1 | 0000008858-21-000048 | NetIncomeLoss | us-gaap/2020 |     nan | 20210930 |      1 | USD   |  1.11318e+08 |        nan |
|  2 | 0000008947-21-000181 | NetIncomeLoss | us-gaap/2020 |     nan | 20200831 |      1 | USD   | -1.79e+06    |        nan |
|  3 | 0000008947-21-000181 | NetIncomeLoss | us-gaap/2020 |     nan | 20210831 |      1 | USD   |  1.8978e+07  |        nan |
|  4 | 0000008947-21-000181 | NetIncomeLoss | us-gaap/2020 |     nan | 20210831 |      2 | USD   |  4.1315e+07 

In [33]:
print(tabulate(dfPre.head(100), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+----------------------+----------+--------+--------+---------+---------+---------------+--------------+---------------------------------------------------------------------+------------+
|    | adsh                 |   report |   line | stmt   |   inpth | rfile   | tag           | version      | plabel                                                              |   negating |
|----+----------------------+----------+--------+--------+---------+---------+---------------+--------------+---------------------------------------------------------------------+------------|
|  0 | 0001104659-21-136311 |        6 |      2 | CF     |       0 | H       | NetIncomeLoss | us-gaap/2020 | Net income (loss)                                                   |          0 |
|  1 | 0001104659-21-136311 |        5 |     23 | EQ     |       0 | H       | NetIncomeLoss | us-gaap/2020 | Net income (loss)                                                   |          0 |
|  2 | 0000950170-21-003759 |      

In [34]:
print(tabulate(dfSub.head(100), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+----------------------+---------+--------------------------------------------------+-------+-------------+----------+---------------------+------------+------------------------------------------+-------------------------------------+------------------+-------------+----------+---------------------+------------+------------------------------------------+---------------------------------+--------------+-----------+-------------+--------------------------------------------+---------------+-------+--------+-------+--------+-------------+------+------+----------+-----------------------+-----------+----------+----------------------------------+---------+---------+
|    | adsh                 |     cik | name                                             |   sic | countryba   | stprba   | cityba              | zipba      | bas1                                     | bas2                                | baph             | countryma   | stprma   | cityma              | zipma      | mas1    

In [35]:
print(tabulate(dfTag.head(100), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+-----------------------------------------------------------------------------------------------------+--------------+----------+------------+------------+--------+--------+---------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|    | tag                                                                                                 | ver

In [36]:
print(tabulate(dfSym.head(10), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+----------+---------+
|    | symbol   |     cik |
|----+----------+---------|
|  0 | aapl     |  320193 |
|  1 | msft     |  789019 |
|  2 | brk-b    | 1067983 |
|  3 | unh      |  731766 |
|  4 | jnj      |  200406 |
|  5 | v        | 1403161 |
|  6 | tsm      | 1046179 |
|  7 | xom      |   34088 |
|  8 | wmt      |  104169 |
|  9 | spy      |  884394 |
+----+----------+---------+
adsh         object
tag          object
version      object
coreg        object
ddate         int64
qtrs          int64
uom          object
value       float64
footnote     object
dtype: object


In [37]:
print(dfNum.iloc[0])
print(dfNum.columns.values)

adsh        0000008858-21-000048
tag                NetIncomeLoss
version             us-gaap/2020
coreg                        NaN
ddate                   20200930
qtrs                           1
uom                          USD
value                -18889000.0
footnote                     NaN
Name: 0, dtype: object
['adsh' 'tag' 'version' 'coreg' 'ddate' 'qtrs' 'uom' 'value' 'footnote']


In [38]:
from datetime import date, datetime, timedelta
from marshmallow import Schema, fields

class FinancialElementImportDto:
    label = ""          #Tag.doc
    concept = ""        #Num.tag
    info = ""           #Pre.plabel
    unit = ""           #Num.uom
    value = 0.0         #Num.value

class FinancialsDataDto:
    bs = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    cf = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    ic = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    

class SymbolFinancialsDto:
    startDate = date.today()   #Sub.period 
    endDate = date.today()     #Sub.period + Sub.fp
    year = 0                   #Sub.fy
    quarter = ""               #Sub.fp
    symbol = ""                #Sub.cik -> Sym.cik -> Sym.symbol
    name = ""                  #Sub.name
    country = ""               #Sub.countryma
    city = ""                  #Sub.cityma
    data = FinancialsDataDto()
    
class FinancialElementImportSchema(Schema):
    label = fields.String()
    concept = fields.String()
    info = fields.String()
    unit = fields.String()
    value = fields.Int()
    
class FinancialsDataSchema(Schema):
    bs = fields.List(fields.Nested(FinancialElementImportSchema()))
    cf = fields.List(fields.Nested(FinancialElementImportSchema()))
    ic = fields.List(fields.Nested(FinancialElementImportSchema()))
    
class SymbolFinancialsSchema(Schema):
    startDate = fields.DateTime()
    endDate = fields.DateTime()
    year = fields.Int()
    quarter = fields.String()
    symbol = fields.String()
    name = fields.String()
    country = fields.String()
    city = fields.String()
    data = fields.Nested(FinancialsDataSchema)

In [39]:
import os

#isExist = os.path.exists("exportFiles/"+dirname)
isExist = 1 == 2
if isExist:
    for filename in os.listdir("exportFiles/"+dirname):
        with open(os.path.join("exportFiles/"+dirname, filename), 'r') as file:
            jsonStr = file.read()
            file.close()
        jsonStr = jsonStr.replace("'", '"').replace(", ",",").replace('\\r', '').replace('\\n', ' ')
        jsonStr = ''.join(jsonStr.splitlines())
        with open(os.path.join("exportFiles/"+dirname, filename), 'w') as file:    
            file.write(jsonStr)
            file.close()
    

# Create a single SymbolFinancialsDto

In [ ]:
#json_string = json.dumps([ob.__dict__ for ob in list_name])
#rslt_df = dataframe[dataframe['Percentage'] > 80]
import numpy as np
from pprint import pprint
import os
from dateutil.relativedelta import relativedelta
import json

def npInt_to_str(var):
    return str(list(np.reshape(np.asarray(var), (1, np.size(var)))[0]))[1:-1]

def formatDateNpNum(var):
    dateStr = npInt_to_str(var)
    return dateStr[0:4]+"-"+dateStr[4:6]+"-"+dateStr[6:8]

fileStartTime = datetime.now()
for subId in range(len(dfSub)):
    startTime = datetime.now()
    submitted = dfSub.iloc[subId]
    sfDto = SymbolFinancialsDto()
    sfDto.data = FinancialsDataDto()
    sfDto.data.bs = []
    sfDto.data.cf = []
    sfDto.data.ic = []
    try:
        periodStartDate = date.fromisoformat(formatDateNpNum(submitted["period"]))
    except ValueError:
        print("File exportFiles/"+dirname+"/"+submitted["adsh"]+".json has Period: "+str(submitted["period"]))
        continue       
    sfDto.startDate = periodStartDate
    sfDto.endDate = date.today() #default value
    if submitted["fy"] == np.nan or np.isnan(submitted["fy"]):
        sfDto.year = 0
    else: 
        sfDto.year = submitted["fy"].astype(int)        
    sfDto.quarter = str(submitted["fp"]).strip().upper()
    
    if sfDto.quarter == "FY" or sfDto.quarter == "CY":
        sfDto.endDate = periodStartDate + relativedelta(months=+12, days=-1)
    elif sfDto.quarter == "H1" or sfDto.quarter == "H2":
        sfDto.endDate = periodStartDate + relativedelta(months=+6, days=-1)
    elif sfDto.quarter == "T1" or sfDto.quarter == "T2" or sfDto.quarter == "T3":
        sfDto.endDate = periodStartDate + relativedelta(months=+4, days=-1)
    elif sfDto.quarter == "Q1" or sfDto.quarter == "Q2" or sfDto.quarter == "Q3" or sfDto.quarter == "Q4":
        sfDto.endDate = periodStartDate + relativedelta(months=+3, days=-1)
    else:
        continue
        
    #symStr = dfSym[dfSym["cik"]==submitted.cik].symbol.str.upper().astype("string")
    #print(symStr)
    val = dfSym[dfSym["cik"]==submitted["cik"]]
    if len(val) > 0:
        sfDto.symbol = val["symbol"].to_string(index = False).split("\n")[0].split("\\n")[0].strip().split(" ")[0].strip().upper()
        if len(sfDto.symbol) > 19 or len(sfDto.symbol) < 1:
            print("File exportFiles/"+dirname+"/"+submitted["adsh"]+".json has Symbol "+sfDto.symbol)
    else:
        print("File exportFiles/"+dirname+"/"+submitted["adsh"]+".json has Symbol "+val["symbol"].to_string(index = False))
        continue
        
    sfDto.name = submitted["name"]
    sfDto.country = submitted["countryma"]
    sfDto.city = submitted["cityma"]

    sdDto = FinancialsDataDto()
    #print(filteredDfPre)
    dfNum['value'] = pd.to_numeric(dfNum['value'], errors='coerce')
    dfNum = dfNum.dropna(subset=['value'])
    dfNum['value'] = dfNum['value'].astype(int)
    #filteredDfNum = dfNum[dfNum['adsh'] == submitted['adsh']]       
    filteredDfNum = dfNum.loc[dfNum['adsh'].values == submitted['adsh']]
    filteredDfNum.reset_index()

    for myId in range(len(filteredDfNum)): 
        #print(myId)
        #print(filteredDfNum.iloc[myId])
        myNum = filteredDfNum.iloc[myId]
        myDto = FinancialElementImportDto()
        myTag = dfTag[dfTag["tag"] == myNum['tag']]
        myDto.label = myTag["doc"].to_string(index = False)
        myDto.concept = myNum["tag"]
        #myPre = dfPre[(dfPre['adsh'] == submitted["adsh"]) & (dfPre['tag'] == myNum['tag'])]
        myPre = dfPre.loc[np.where((dfPre['adsh'].values == submitted["adsh"]) & (dfPre['tag'].values == myNum['tag']))]
        myDto.info = myPre["plabel"].to_string(index = False).replace('"',"'")
        myDto.unit = myNum["uom"]
        #print(myNum["value"])        
        myDto.value = myNum["value"]
        #print(myDto.__dict__)
        FinancialElementImportSchema().dump(myDto)
        if myPre['stmt'].to_string(index = False).strip() == 'BS':
            sfDto.data.bs.append(myDto)
        elif myPre['stmt'].to_string(index = False).strip() == 'CF':
            sfDto.data.cf.append(myDto)
        elif myPre['stmt'].to_string(index = False).strip() == 'IC':
            sfDto.data.ic.append(myDto)

    #print(len(filteredDfNum))
    #print(submitted)
    #print(sfDto.__dict__)    
    #json.dumps(sfDto.__dict__)    
    marshmellowStart = datetime.now()
    result = SymbolFinancialsSchema().dump(sfDto)
    result = str(json.dumps(result))
    #result = result.replace('"','öäü').replace("'", '"').replace("öäü","'").replace(", ",",")
    result = result.replace('\\r', '').replace('\\n', ' ')
    
    print("FilterDfNum size: "+str(len(filteredDfNum))+" Dtos size: "+str((len(sfDto.data.bs)+len(sfDto.data.cf)+len(sfDto.data.ic)))+" Bs: "+str(len(sfDto.data.bs))+" Cf: "+str(len(sfDto.data.cf))+" Ic: "+str(len(sfDto.data.ic))+" Json size: "+str(int(len(str(result)) / 1024))+"kb.")
    
    #print("Json size "+str(int(len(str(result)) / 1024))+"kb. Time: "+str((datetime.now() - marshmellowStart) / timedelta(milliseconds=1))+"ms.")
    
    #print(result)
    #pprint(result)    
    isExist = os.path.exists("exportFiles/"+dirname)
    if not isExist:
        os.makedirs("exportFiles/"+dirname)
    json_file = open("exportFiles/"+dirname+"/"+submitted["adsh"]+".json","w")
    json_file.write(result)
    json_file.close()
    endTime = datetime.now()
    print("file "+str(subId+1)+" of "+str(len(dfSub)+1)+" exportFiles/"+dirname+"/"+submitted["adsh"]+".json stored in "+str((endTime - startTime) / timedelta(milliseconds=1))+"ms.")
    
fileEndTime = datetime.now()
print("Time "+str((fileEndTime - fileStartTime) / timedelta(seconds=1))+"sec for "+str(len(dfSub)+1)+" files.")

File exportFiles/2021q4/0001096906-21-003094.json has Symbol Series([], )
FilterDfNum size: 2 Dtos size: 0 Bs: 0 Cf: 0 Ic: 0 Json size: 0kb.
file 16 of 24093 exportFiles/2021q4/0001104659-21-154702.json stored in 566.417ms.
FilterDfNum size: 198 Dtos size: 66 Bs: 66 Cf: 0 Ic: 0 Json size: 15kb.
file 45 of 24093 exportFiles/2021q4/0001178913-21-003937.json stored in 14296.281ms.
File exportFiles/2021q4/0001193125-21-369778.json has Symbol Series([], )
File exportFiles/2021q4/0001193125-21-369787.json has Symbol Series([], )
FilterDfNum size: 286 Dtos size: 105 Bs: 73 Cf: 32 Ic: 0 Json size: 25kb.
file 58 of 24093 exportFiles/2021q4/0001193125-21-370461.json stored in 20316.779ms.
FilterDfNum size: 130 Dtos size: 74 Bs: 46 Cf: 28 Ic: 0 Json size: 38kb.
file 72 of 24093 exportFiles/2021q4/0001193125-21-370643.json stored in 9195.325ms.
FilterDfNum size: 75 Dtos size: 39 Bs: 20 Cf: 19 Ic: 0 Json size: 16kb.
file 74 of 24093 exportFiles/2021q4/0001193125-21-370650.json stored in 5077.562ms.

FilterDfNum size: 115 Dtos size: 48 Bs: 40 Cf: 8 Ic: 0 Json size: 25kb.
file 432 of 24093 exportFiles/2021q4/0001213900-21-067764.json stored in 4877.829ms.
FilterDfNum size: 111 Dtos size: 46 Bs: 21 Cf: 25 Ic: 0 Json size: 44kb.
file 444 of 24093 exportFiles/2021q4/0001410578-21-000609.json stored in 4939.697ms.
FilterDfNum size: 311 Dtos size: 120 Bs: 71 Cf: 49 Ic: 0 Json size: 31kb.
file 449 of 24093 exportFiles/2021q4/0001493152-21-032565.json stored in 13486.361ms.
FilterDfNum size: 59 Dtos size: 35 Bs: 30 Cf: 5 Ic: 0 Json size: 8kb.
file 453 of 24093 exportFiles/2021q4/0001493152-21-032579.json stored in 2918.543ms.
FilterDfNum size: 317 Dtos size: 104 Bs: 68 Cf: 36 Ic: 0 Json size: 25kb.
file 465 of 24093 exportFiles/2021q4/0001562762-21-000500.json stored in 13394.176ms.
FilterDfNum size: 267 Dtos size: 111 Bs: 70 Cf: 41 Ic: 0 Json size: 25kb.
file 476 of 24093 exportFiles/2021q4/0001640334-21-003302.json stored in 14235.591ms.
FilterDfNum size: 314 Dtos size: 97 Bs: 44 Cf: 53 

FilterDfNum size: 395 Dtos size: 144 Bs: 79 Cf: 65 Ic: 0 Json size: 34kb.
file 868 of 24093 exportFiles/2021q4/0001554795-21-000422.json stored in 27815.989ms.
FilterDfNum size: 310 Dtos size: 114 Bs: 64 Cf: 50 Ic: 0 Json size: 31kb.
file 870 of 24093 exportFiles/2021q4/0001558370-21-017007.json stored in 21890.946ms.
FilterDfNum size: 289 Dtos size: 104 Bs: 51 Cf: 53 Ic: 0 Json size: 41kb.
file 874 of 24093 exportFiles/2021q4/0001562762-21-000494.json stored in 20388.185ms.
File exportFiles/2021q4/0001640334-21-003279.json has Symbol Series([], )
File exportFiles/2021q4/0001640334-21-003281.json has Symbol Series([], )
FilterDfNum size: 216 Dtos size: 79 Bs: 41 Cf: 38 Ic: 0 Json size: 18kb.
file 906 of 24093 exportFiles/2021q4/0001683168-21-006551.json stored in 9786.544ms.
FilterDfNum size: 220 Dtos size: 122 Bs: 68 Cf: 54 Ic: 0 Json size: 33kb.
file 907 of 24093 exportFiles/2021q4/0001683168-21-006560.json stored in 8826.717ms.
FilterDfNum size: 97 Dtos size: 56 Bs: 38 Cf: 18 Ic: 0 

FilterDfNum size: 68 Dtos size: 34 Bs: 17 Cf: 17 Ic: 0 Json size: 40kb.
file 1242 of 24093 exportFiles/2021q4/0001104659-21-152116.json stored in 3129.728ms.
FilterDfNum size: 60 Dtos size: 25 Bs: 15 Cf: 10 Ic: 0 Json size: 18kb.
file 1249 of 24093 exportFiles/2021q4/0001104659-21-152183.json stored in 3111.557ms.
FilterDfNum size: 396 Dtos size: 143 Bs: 85 Cf: 58 Ic: 0 Json size: 35kb.
file 1259 of 24093 exportFiles/2021q4/0001104659-21-152249.json stored in 15582.554ms.
FilterDfNum size: 67 Dtos size: 38 Bs: 20 Cf: 18 Ic: 0 Json size: 19kb.
file 1279 of 24093 exportFiles/2021q4/0001193125-21-362728.json stored in 3084.461ms.
FilterDfNum size: 102 Dtos size: 61 Bs: 52 Cf: 9 Ic: 0 Json size: 20kb.
file 1280 of 24093 exportFiles/2021q4/0001193125-21-362740.json stored in 4570.438ms.
File exportFiles/2021q4/0001193125-21-362797.json has Symbol Series([], )
FilterDfNum size: 368 Dtos size: 202 Bs: 101 Cf: 101 Ic: 0 Json size: 73kb.
file 1293 of 24093 exportFiles/2021q4/0001193125-21-36302

FilterDfNum size: 100 Dtos size: 48 Bs: 28 Cf: 20 Ic: 0 Json size: 11kb.
file 1686 of 24093 exportFiles/2021q4/0001477932-21-009362.json stored in 7553.084ms.
FilterDfNum size: 189 Dtos size: 99 Bs: 61 Cf: 38 Ic: 0 Json size: 26kb.
file 1692 of 24093 exportFiles/2021q4/0001493152-21-031970.json stored in 13738.148ms.
FilterDfNum size: 287 Dtos size: 108 Bs: 61 Cf: 47 Ic: 0 Json size: 30kb.
file 1700 of 24093 exportFiles/2021q4/0001493152-21-031993.json stored in 20751.65ms.
FilterDfNum size: 294 Dtos size: 110 Bs: 58 Cf: 52 Ic: 0 Json size: 30kb.
file 1701 of 24093 exportFiles/2021q4/0001493152-21-031995.json stored in 21213.115ms.
FilterDfNum size: 274 Dtos size: 100 Bs: 56 Cf: 44 Ic: 0 Json size: 23kb.
file 1703 of 24093 exportFiles/2021q4/0001493152-21-032013.json stored in 19411.244ms.
FilterDfNum size: 204 Dtos size: 89 Bs: 56 Cf: 33 Ic: 0 Json size: 21kb.
file 1704 of 24093 exportFiles/2021q4/0001493152-21-032020.json stored in 14716.172ms.
FilterDfNum size: 365 Dtos size: 109 Bs

FilterDfNum size: 125 Dtos size: 54 Bs: 32 Cf: 22 Ic: 0 Json size: 29kb.
file 1998 of 24093 exportFiles/2021q4/0001213900-21-066022.json stored in 5162.804ms.
FilterDfNum size: 56 Dtos size: 18 Bs: 12 Cf: 6 Ic: 0 Json size: 15kb.
file 2002 of 24093 exportFiles/2021q4/0001213900-21-066061.json stored in 2502.754ms.
FilterDfNum size: 113 Dtos size: 43 Bs: 25 Cf: 18 Ic: 0 Json size: 24kb.
file 2004 of 24093 exportFiles/2021q4/0001213900-21-066078.json stored in 4788.502ms.
FilterDfNum size: 89 Dtos size: 32 Bs: 14 Cf: 18 Ic: 0 Json size: 32kb.
file 2005 of 24093 exportFiles/2021q4/0001213900-21-066082.json stored in 3797.137ms.
FilterDfNum size: 141 Dtos size: 57 Bs: 32 Cf: 25 Ic: 0 Json size: 53kb.
file 2006 of 24093 exportFiles/2021q4/0001213900-21-066083.json stored in 6362.81ms.
File exportFiles/2021q4/0001359687-21-000015.json has Symbol Series([], )
FilterDfNum size: 66 Dtos size: 27 Bs: 17 Cf: 10 Ic: 0 Json size: 19kb.
file 2018 of 24093 exportFiles/2021q4/0001410578-21-000546.json

FilterDfNum size: 354 Dtos size: 138 Bs: 74 Cf: 64 Ic: 0 Json size: 31kb.
file 2417 of 24093 exportFiles/2021q4/0001683168-21-006416.json stored in 14950.974ms.
FilterDfNum size: 83 Dtos size: 54 Bs: 36 Cf: 18 Ic: 0 Json size: 13kb.
file 2419 of 24093 exportFiles/2021q4/0001683168-21-006422.json stored in 3602.451ms.
FilterDfNum size: 184 Dtos size: 68 Bs: 34 Cf: 34 Ic: 0 Json size: 18kb.
file 2423 of 24093 exportFiles/2021q4/0001779372-21-000002.json stored in 7778.702ms.
FilterDfNum size: 51 Dtos size: 33 Bs: 27 Cf: 6 Ic: 0 Json size: 7kb.
file 2429 of 24093 exportFiles/2021q4/0001829126-21-016607.json stored in 2415.955ms.
FilterDfNum size: 60 Dtos size: 26 Bs: 18 Cf: 8 Ic: 0 Json size: 6kb.
file 2430 of 24093 exportFiles/2021q4/0001829126-21-016613.json stored in 2685.484ms.
FilterDfNum size: 365 Dtos size: 116 Bs: 59 Cf: 57 Ic: 0 Json size: 28kb.
file 2432 of 24093 exportFiles/2021q4/0001851734-21-000150.json stored in 16857.42ms.
FilterDfNum size: 364 Dtos size: 142 Bs: 76 Cf: 66